<a href="https://colab.research.google.com/github/R91198/Fashion-MNIST/blob/master/Optimization_with_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=950bad0c8e11eddfff16554d7f5d6f607847a5837e2510ae5f4af31752261575
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=c09b815200b35ff7e49a4098ba14e7fd370f9c1d57971058ab5b145cb7b9e0b1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.5561 - accuracy: 0.8175 - val_loss: 0.4032 - val_accuracy: 0.8498
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3732 - accuracy: 0.8621 - val_loss: 0.3651 - val_accuracy: 0.8645
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3506 - accuracy: 0.8712 - val_loss: 0.3736 - val_accuracy: 0.8647


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4004 - accuracy: 0.8567 - val_loss: 0.3442 - val_accuracy: 0.8722
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2603 - accuracy: 0.9042 - val_loss: 0.2887 - val_accuracy: 0.8952
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2014 - accuracy: 0.9245 - val_loss: 0.2436 - val_accuracy: 0.9138


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3845 - accuracy: 0.8609 - val_loss: 0.3067 - val_accuracy: 0.8872
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2426 - accuracy: 0.9103 - val_loss: 0.2478 - val_accuracy: 0.9112
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1840 - accuracy: 0.9302 - val_loss: 0.2942 - val_accuracy: 0.8957


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5361 - accuracy: 0.8056 - val_loss: 0.4295 - val_accuracy: 0.8387
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4030 - accuracy: 0.8500 - val_loss: 0.4175 - val_accuracy: 0.8512
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3638 - accuracy: 0.8648 - val_loss: 0.3992 - val_accuracy: 0.8627


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4613 - accuracy: 0.8382 - val_loss: 0.3870 - val_accuracy: 0.8577
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3345 - accuracy: 0.8759 - val_loss: 0.3223 - val_accuracy: 0.8797
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2907 - accuracy: 0.8904 - val_loss: 0.3720 - val_accuracy: 0.8658


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1115184   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,173,802
Trainable params: 1,173,802
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.7, initial_epoch=3)

Epoch 4/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0107 - accuracy: 0.9971 - val_loss: 0.3329 - val_accuracy: 0.9609
Epoch 5/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0072 - accuracy: 0.9979 - val_loss: 0.3251 - val_accuracy: 0.9642
Epoch 6/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0114 - accuracy: 0.9967 - val_loss: 0.2765 - val_accuracy: 0.9648
Epoch 7/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0070 - accuracy: 0.9980 - val_loss: 0.3570 - val_accuracy: 0.9600
Epoch 8/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0034 - accuracy: 0.9984 - val_loss: 0.3454 - val_accuracy: 0.9649
Epoch 9/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 0.3398 - val_accuracy: 0.9632
Epoch 10/10
563/563 [==============================] - 5s 9ms/step - loss: 0.0040 - accuracy: 0.9987 - val_loss: 0.3304 - val_accuracy: 0.9663
